In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier

In [2]:
from sklearn.model_selection import train_test_split

train = pd.read_csv('..\\data\\UCI-fault\\Faults.csv',header=None)
pd.set_option('display.max_columns', None)
train.columns = ["X_Minimum", "X_Maximum", "Y_Minimum", "Y_Maximum", "Pixels_Areas", "X_Perimeter", "Y_Perimeter",
              "Sum_of_Luminosity", "Minimum_of_Luminosity", "Maximum_of_Luminosity", "Length_of_Conveyer", "TypeOfSteel_A300",
              "TypeOfSteel_A400", "Steel_Plate_Thickness", "Edges_Index", "Empty_Index", "Square_Index", "Outside_X_Index",
              "Edges_X_Index", "Edges_Y_Index", "Outside_Global_Index", "LogOfAreas", "Log_X_Index", "Log_Y_Index", "Orientation_Index",
              "Luminosity_Index", "SigmoidOfAreas", "Pastry", "Z_Scratch", "K_Scatch", "Stains", "Dirtiness", "Bumps", "Other_Faults" ]
train

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,42,50,270900,270944,267,17,44,24220,76,108,1687,1,0,80,0.0498,0.2415,0.1818,0.0047,0.4706,1.0000,1.0,2.4265,0.9031,1.6435,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,1687,1,0,80,0.7647,0.3793,0.2069,0.0036,0.6000,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,1623,1,0,100,0.9710,0.3426,0.3333,0.0037,0.7500,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,853,860,369370,369415,176,13,45,18996,99,126,1353,0,1,290,0.7287,0.4413,0.1556,0.0052,0.5385,1.0000,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,1289,1306,498078,498335,2409,60,260,246930,37,126,1353,0,1,185,0.0695,0.4486,0.0662,0.0126,0.2833,0.9885,1.0,3.3818,1.2305,2.4099,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,249,277,325780,325796,273,54,22,35033,119,141,1360,0,1,40,0.3662,0.3906,0.5714,0.0206,0.5185,0.7273,0.0,2.4362,1.4472,1.2041,-0.4286,0.0026,0.7254,0,0,0,0,0,0,1
1937,144,175,340581,340598,287,44,24,34599,112,133,1360,0,1,40,0.2118,0.4554,0.5484,0.0228,0.7046,0.7083,0.0,2.4579,1.4914,1.2305,-0.4516,-0.0582,0.8173,0,0,0,0,0,0,1
1938,145,174,386779,386794,292,40,22,37572,120,140,1360,0,1,40,0.2132,0.3287,0.5172,0.0213,0.7250,0.6818,0.0,2.4654,1.4624,1.1761,-0.4828,0.0052,0.7079,0,0,0,0,0,0,1
1939,137,170,422497,422528,419,97,47,52715,117,140,1360,0,1,40,0.2015,0.5904,0.9394,0.0243,0.3402,0.6596,0.0,2.6222,1.5185,1.4914,-0.0606,-0.0171,0.9919,0,0,0,0,0,0,1


In [3]:
y_value = pd.DataFrame(train, columns=["Pastry", "Z_Scratch", "K_Scatch", "Stains", "Dirtiness", "Bumps", "Other_Faults"])
x_value = pd.DataFrame(train, columns=["X_Minimum", "X_Maximum", "Y_Minimum", "Y_Maximum", "Pixels_Areas", "X_Perimeter", "Y_Perimeter",
              "Sum_of_Luminosity", "Minimum_of_Luminosity", "Maximum_of_Luminosity", "Length_of_Conveyer", "TypeOfSteel_A300",
              "TypeOfSteel_A400", "Steel_Plate_Thickness", "Edges_Index", "Empty_Index", "Square_Index", "Outside_X_Index",
              "Edges_X_Index", "Edges_Y_Index", "Outside_Global_Index", "LogOfAreas", "Log_X_Index", "Log_Y_Index", "Orientation_Index",
              "Luminosity_Index", "SigmoidOfAreas"])

#x, x_test, y, y_test = train_test_split(x_value,y_value, test_size=0.2, shuffle=True, random_state=1004)
#x_train, x_valid, y_train, y_valid = train_test_split(x,y, test_size=0.11, shuffle=True, random_state=1004)

In [4]:
tmp=[]
target_columns = ["Pastry", "Z_Scratch", "K_Scatch", "Stains", "Dirtiness", "Bumps", "Other_Faults"]
for i in range(len(y_value)):
    for j in range(0,7):
        if y_value.loc[i][j] == 1:
            tmp.append(target_columns[j])
targets = pd.DataFrame(tmp,columns=["target"])
x_value['target'] = tmp
x_value
'''
target = pd.DataFrame(tmp)
x, x_test, y, y_test = train_test_split(x_value,target, test_size=0.2, shuffle=True, random_state=1004)
x_train, x_valid, y_train, y_valid = train_test_split(x,y, test_size=0.11, shuffle=True, random_state=1004)

print(x_train)
'''
train = x_value

In [5]:
train

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,target
0,42,50,270900,270944,267,17,44,24220,76,108,1687,1,0,80,0.0498,0.2415,0.1818,0.0047,0.4706,1.0000,1.0,2.4265,0.9031,1.6435,0.8182,-0.2913,0.5822,Pastry
1,645,651,2538079,2538108,108,10,30,11397,84,123,1687,1,0,80,0.7647,0.3793,0.2069,0.0036,0.6000,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984,Pastry
2,829,835,1553913,1553931,71,8,19,7972,99,125,1623,1,0,100,0.9710,0.3426,0.3333,0.0037,0.7500,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.2150,Pastry
3,853,860,369370,369415,176,13,45,18996,99,126,1353,0,1,290,0.7287,0.4413,0.1556,0.0052,0.5385,1.0000,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212,Pastry
4,1289,1306,498078,498335,2409,60,260,246930,37,126,1353,0,1,185,0.0695,0.4486,0.0662,0.0126,0.2833,0.9885,1.0,3.3818,1.2305,2.4099,0.9338,-0.1992,1.0000,Pastry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,249,277,325780,325796,273,54,22,35033,119,141,1360,0,1,40,0.3662,0.3906,0.5714,0.0206,0.5185,0.7273,0.0,2.4362,1.4472,1.2041,-0.4286,0.0026,0.7254,Other_Faults
1937,144,175,340581,340598,287,44,24,34599,112,133,1360,0,1,40,0.2118,0.4554,0.5484,0.0228,0.7046,0.7083,0.0,2.4579,1.4914,1.2305,-0.4516,-0.0582,0.8173,Other_Faults
1938,145,174,386779,386794,292,40,22,37572,120,140,1360,0,1,40,0.2132,0.3287,0.5172,0.0213,0.7250,0.6818,0.0,2.4654,1.4624,1.1761,-0.4828,0.0052,0.7079,Other_Faults
1939,137,170,422497,422528,419,97,47,52715,117,140,1360,0,1,40,0.2015,0.5904,0.9394,0.0243,0.3402,0.6596,0.0,2.6222,1.5185,1.4914,-0.0606,-0.0171,0.9919,Other_Faults


In [6]:
if "Set" not in train.columns:
    train["Set"] = np.random.choice(["train","valid","test"], p = [.8, .1, .1], size = (train.shape[0],))

train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index
test_indices = train[train.Set=="test"].index

In [7]:
nunique = train.nunique()
types = train.dtypes

categorical_columns = []
categorical_dims =  {}
for col in train.columns:
    if types[col] == 'object' or nunique[col] < 200:
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = train[col].fillna("VV_likely")
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        train.fillna(train.loc[train_indices, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
unused_feat = ['Set']
features = ["X_Minimum", "X_Maximum", "Y_Minimum", "Y_Maximum", "Pixels_Areas", "X_Perimeter", "Y_Perimeter",
              "Sum_of_Luminosity", "Minimum_of_Luminosity", "Maximum_of_Luminosity", "Length_of_Conveyer", "TypeOfSteel_A300",
              "TypeOfSteel_A400", "Steel_Plate_Thickness", "Edges_Index", "Empty_Index", "Square_Index", "Outside_X_Index",
              "Edges_X_Index", "Edges_Y_Index", "Outside_Global_Index", "LogOfAreas", "Log_X_Index", "Log_Y_Index", "Orientation_Index",
              "Luminosity_Index", "SigmoidOfAreas"]
cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]



X_train = train[features].values[train_indices]
y_train = train['target'].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train['target'].values[valid_indices]

X_test = train[features].values[test_indices]
y_test = train['target'].values[test_indices]

Minimum_of_Luminosity 161
Maximum_of_Luminosity 100
Length_of_Conveyer 84
TypeOfSteel_A300 2
TypeOfSteel_A400 2
Steel_Plate_Thickness 24
Outside_Global_Index 3
Log_X_Index 183
target 7
Set 3


In [14]:
print(X_train)
print(X_train.shape)
print(type(X_train))

print(X_valid)
print(X_valid.shape)
print(type(X_valid))

print(X_test)
print(X_test.shape)
print(type(X_test))
print()
print()

print(y_train)
print(y_train.shape)
print(type(y_train))

print(y_valid)
print(y_valid.shape)
print(type(y_valid))

print(y_test)
print(y_test.shape)
print(type(y_test))

[[ 4.200000e+01  5.000000e+01  2.709000e+05 ...  8.182000e-01
  -2.913000e-01  5.822000e-01]
 [ 8.290000e+02  8.350000e+02  1.553913e+06 ...  6.667000e-01
  -1.228000e-01  2.150000e-01]
 [ 8.530000e+02  8.600000e+02  3.693700e+05 ...  8.444000e-01
  -1.568000e-01  5.212000e-01]
 ...
 [ 1.440000e+02  1.750000e+02  3.405810e+05 ... -4.516000e-01
  -5.820000e-02  8.173000e-01]
 [ 1.450000e+02  1.740000e+02  3.867790e+05 ... -4.828000e-01
   5.200000e-03  7.079000e-01]
 [ 1.370000e+02  1.700000e+02  4.224970e+05 ... -6.060000e-02
  -1.710000e-02  9.919000e-01]]
(1529, 27)
<class 'numpy.ndarray'>
[[ 6.450000e+02  6.510000e+02  2.538079e+06 ...  7.931000e-01
  -1.756000e-01  2.984000e-01]
 [ 4.300000e+02  4.410000e+02  1.002500e+05 ...  8.736000e-01
  -2.267000e-01  9.874000e-01]
 [ 1.328000e+03  1.338000e+03  2.691161e+06 ...  6.429000e-01
  -2.785000e-01  4.630000e-01]
 ...
 [ 1.053000e+03  1.063000e+03  1.732080e+05 ...  4.444000e-01
   1.500000e-03  3.068000e-01]
 [ 2.490000e+02  2.77000

In [10]:
cat_idxs

[8, 9, 10, 11, 12, 13, 20, 22]

In [11]:
cat_dims

[161, 100, 84, 2, 2, 24, 3, 183]

In [12]:
tabnet_params = {"cat_idxs":cat_idxs,
                 "cat_dims":cat_dims,
                 "cat_emb_dim":1,
                 "optimizer_fn":torch.optim.Adam,
                 "optimizer_params":dict(lr=2e-2),
                 "scheduler_params":{"step_size":50, # how to use learning rate scheduler
                                 "gamma":0.9},
                 "scheduler_fn":torch.optim.lr_scheduler.StepLR,
                 "mask_type":'entmax' # "sparsemax"
                }

clf = TabNetClassifier(**tabnet_params
)

Device used : cuda


In [19]:
'''
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier

tabnet_params = {"cat_idxs":cat_idxs,
                 "cat_dims":cat_dims,
                 "cat_emb_dim":1,
                 "optimizer_fn":torch.optim.Adam,
                 "optimizer_params":dict(lr=2e-2),
                 "scheduler_params":{"step_size":50, # how to use learning rate scheduler
                                 "gamma":0.9},
                 "scheduler_fn":torch.optim.lr_scheduler.StepLR,
                 "mask_type":'entmax' # "sparsemax"
                }

clf = TabNetMultiTaskClassifier(**tabnet_params
                      )

'''


'\nfrom pytorch_tabnet.multitask import TabNetMultiTaskClassifier\n\ntabnet_params = {"cat_idxs":cat_idxs,\n                 "cat_dims":cat_dims,\n                 "cat_emb_dim":1,\n                 "optimizer_fn":torch.optim.Adam,\n                 "optimizer_params":dict(lr=2e-2),\n                 "scheduler_params":{"step_size":50, # how to use learning rate scheduler\n                                 "gamma":0.9},\n                 "scheduler_fn":torch.optim.lr_scheduler.StepLR,\n                 "mask_type":\'entmax\' # "sparsemax"\n                }\n\nclf = TabNetMultiTaskClassifier(**tabnet_params\n                      )\n\n'

In [13]:
max_epochs = 100

# This illustrates the warm_start=False behaviour
#save_history = []

clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_name=['train', 'valid'],
        eval_metric=['accuracy'],
        max_epochs=max_epochs , patience=20,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        weights=1,
        drop_last=False
    )
#save_history.append(clf.history["valid_auc"])
    
#assert(np.all(np.array(save_history[0]==np.array(save_history[1]))))

epoch 0  | loss: 2.33768 | train_accuracy: 0.21779 | valid_accuracy: 0.24057 |  0:00:01s
epoch 1  | loss: 2.05953 | train_accuracy: 0.22041 | valid_accuracy: 0.24528 |  0:00:01s
epoch 2  | loss: 1.92698 | train_accuracy: 0.22041 | valid_accuracy: 0.18396 |  0:00:01s
epoch 3  | loss: 1.78222 | train_accuracy: 0.20078 | valid_accuracy: 0.16981 |  0:00:01s
epoch 4  | loss: 1.65191 | train_accuracy: 0.1877  | valid_accuracy: 0.14623 |  0:00:01s
epoch 5  | loss: 1.62553 | train_accuracy: 0.23741 | valid_accuracy: 0.21226 |  0:00:01s
epoch 6  | loss: 1.56963 | train_accuracy: 0.26357 | valid_accuracy: 0.24057 |  0:00:02s
epoch 7  | loss: 1.46293 | train_accuracy: 0.26619 | valid_accuracy: 0.26887 |  0:00:02s
epoch 8  | loss: 1.36747 | train_accuracy: 0.25965 | valid_accuracy: 0.27358 |  0:00:02s
epoch 9  | loss: 1.25583 | train_accuracy: 0.28123 | valid_accuracy: 0.31604 |  0:00:02s
epoch 10 | loss: 1.18312 | train_accuracy: 0.30739 | valid_accuracy: 0.3066  |  0:00:02s
epoch 11 | loss: 1.05